Create deadlock.db

In [32]:
import duckdb
import pandas as pd

con = duckdb.connect("c:/Code/Local Code/Deadlock Database/Deadlock_Match_Prediction/deadlock.db")

In [33]:
df = pd.read_csv(r"\Code\Local Code\Deadlock Database\player_df.csv")

In [34]:
df.head()

,account_id,match_id,hero_id,hero_level,player_team,player_kills,player_deaths,player_assists,denies,net_worth,last_hits,team_abandoned,abandoned_time_s,won,p_total_kills,p_total_deaths,p_avg_kd,p_total_matches,p_h_total_matches,p_h_pick_pct
0,356216207,2560778,2,19,1,10,4,14,1,22164,78,False,0.0,True,14521,10311,1.408302,1762,174,9.88
1,356216207,2563975,1,21,0,10,4,12,0,26105,110,False,0.0,True,14521,10311,1.408302,1762,164,9.31
2,356216207,2567161,27,9,0,1,6,4,0,6970,44,False,0.0,False,14521,10311,1.408302,1762,81,4.60
3,356216207,2574453,19,23,1,8,7,14,3,30027,104,False,0.0,False,14521,10311,1.408302,1762,169,9.59
4,356216207,2579485,50,17,0,8,7,0,3,16395,74,False,0.0,False,14521,10311,1.408302,1762,53,3.01


In [35]:
df['account_id'].value_counts()

account_id
925951531     104775
337955472     102060
244471627     101608
451711628      99603
301594065      92598
               ...  
1175819752        76
1563872085        73
1874854061        56
246353833         56
1871135503        56
Name: count, Length: 424, dtype: int64

In [36]:
df['match_id'].value_counts()

match_id
33228355    219
33229871    195
34494378    189
35118078    187
33613285    186
           ... 
33484323      1
33484852      1
33551333      1
33553030      1
33585452      1
Name: count, Length: 332194, dtype: int64

In [37]:
match_counts = (
    df.groupby('match_id')['account_id']
    .count()
    .reset_index(name='player_count')
)
full_matches = match_counts[match_counts['player_count'] == 2]
print(len(full_matches))

37091


In [38]:
(
    df.groupby('match_id')['account_id']
    .count()
    .value_counts()
    .sort_index()
)

account_id
1      61143
2      37091
3      14549
4      23895
5      19487
       ...  
186        2
187        1
189        1
195        1
219        1
Name: count, Length: 161, dtype: int64

In [39]:
sql = con.execute("PRAGMA table_info('matches')").fetchall()
print(sql)

[(0, 'match_id', 'BIGINT', True, None, True), (1, 'account_id', 'BIGINT', True, None, True), (2, 'start_time', 'TIMESTAMP', False, None, False), (3, 'game_mode', 'VARCHAR', False, None, False), (4, 'match_mode', 'VARCHAR', False, None, False), (5, 'duration_s', 'INTEGER', False, None, False), (6, 'winning_team', 'VARCHAR', False, None, False)]


In [41]:
df.groupby(['account_id', 'match_id']).size().value_counts()

1     67997
2     44258
4     30448
5     24580
7     22655
6     19254
9     18820
3     17975
12    13192
15    11233
10    10154
33     8923
11     8455
13     7882
18     6906
16     5926
14     5691
26     5534
17     5410
42     4551
38     4338
8      3850
28     3439
22     3186
20     3026
32     2500
27     2398
24     2378
44     1969
52     1954
51     1953
55     1905
19     1872
21     1682
34     1269
45     1002
40      968
62      810
36      611
30      545
47      300
Name: count, dtype: int64

In [43]:
dupes = (
    df.groupby(['account_id', 'match_id'])
      .size()
      .reset_index(name='count')
      .sort_values(by='count', ascending=False)
)

print(dupes.head(30))

        account_id  match_id  count
313887  1150831835  32448333     62
314414  1150831835  35268297     62
314415  1150831835  35319686     62
314418  1150831835  35320993     62
314352  1150831835  35049317     62
314353  1150831835  35050217     62
314402  1150831835  35212329     62
314416  1150831835  35320349     62
314417  1150831835  35320468     62
314419  1150831835  35322198     62
314404  1150831835  35234395     62
314405  1150831835  35237943     62
314421  1150831835  35323614     62
314422  1150831835  35324936     62
314410  1150831835  35245605     62
314396  1150831835  35141134     62
314397  1150831835  35142034     62
314382  1150831835  35098729     62
314383  1150831835  35099624     62
314400  1150831835  35149992     62
314413  1150831835  35267918     62
314398  1150831835  35144495     62
314399  1150831835  35149081     62
314355  1150831835  35051557     62
314340  1150831835  35004513     62
314341  1150831835  35015583     62
314390  1150831835  35115450

In [40]:
sql = con.execute("SELECT match_id, account_id, COUNT(*) AS dup_count FROM matches GROUP BY match_id, account_id HAVING COUNT(*) > 1;").fetchall()
print(sql)

[]


Drop tables for reset

In [ ]:
def drop_tables(con):
    con.execute("DROP TABLE IF EXISTS matches")
    con.execute("DROP TABLE IF EXISTS player_matches")
    con.execute("DROP TABLE IF EXISTS player_trends")
    con.execute("DROP TABLE IF EXISTS hero_trends")

Create tables

In [ ]:
con.execute("""
CREATE TABLE matches (
    match_id BIGINT PRIMARY KEY,
    start_time TIMESTAMP,
    game_mode INTEGER,
    match_mode INTEGER,
    match_duration_s INTEGER,
    objectives_mask_team0 BIGINT,
    objectives_mask_team1 BIGINT,
    match_result VARCHAR
)
""")

In [ ]:
con.execute("""
CREATE TABLE player_matches (
    account_id BIGINT,
    match_id BIGINT,
    hero_id INTEGER,
    hero_level INTEGER,
    player_team INTEGER,
    player_kills INTEGER,
    player_deaths INTEGER,
    player_assists INTEGER,
    denies INTEGER,
    net_worth BIGINT,
    last_hits INTEGER,
    team_abandoned BOOLEAN,
    abandoned_time_s INTEGER,
    won BOOLEAN,
    
    PRIMARY KEY (account_id, match_id)
)
""")

In [ ]:
con.execute("""
CREATE TABLE player_trends (
    account_id BIGINT,
    match_id BIGINT,
    hero_id INTEGER,
    p_win_pct_3 FLOAT,
    p_win_pct_5 FLOAT,
    p_streak_3 VARCHAR,
    p_streak_5 VARCHAR,
    h_win_pct_3 FLOAT,
    h_win_pct_5 FLOAT,
    h_streak_3 VARCHAR,
    h_streak_5 VARCHAR,

    PRIMARY KEY (account_id, match_id, hero_id)
)
""")

In [ ]:
con.execute("""
CREATE TABLE hero_trends (
    hero_id INTEGER,
    trend_start_date DATE,
    trend_end_date DATE,
    trend_date DATE,
    trend_window_days INTEGER,
    pick_rate FLOAT,
    win_rate FLOAT,
    average_kills FLOAT,
    average_deaths FLOAT,
    average_assists FLOAT,

    PRIMARY KEY (hero_id, trend_start_date, trend_end_date, trend_window_days)
)
""")


In [ ]:
con.execute("""
    CREATE TABLE player_profiles (
    account_id BIGINT PRIMARY KEY,
    
    avg_p_win_pct_3 FLOAT,
    avg_p_win_pct_5 FLOAT,
    avg_h_win_pct_3 FLOAT,
    avg_h_win_pct_5 FLOAT,
    
    total_matches INTEGER,

    norm_avg_p_win_pct_3 FLOAT,  
    norm_avg_p_win_pct_5 FLOAT,
    norm_avg_h_win_pct_3 FLOAT,
    norm_avg_h_win_pct_5 FLOAT
)
""")

duckdb sql checks

In [8]:
con.execute("SELECT COUNT(*) FROM match_holdout").fetchone()[0]

5968

In [ ]:
result = con.execute("SELECT * FROM player_profiles LIMIT 5").fetchdf()
print(result)

In [ ]:
import requests
m_id = [471321,471609,471883]
site = "https://api.deadlock-api.com"
endpoint = "/v1/matches/metadata?"
url=f"{site}{endpoint}include_player_info=true&match_ids={m_id}"
response = requests.get(url)
if response.status_code == 200:
    json_match_data = response.json()
    df_match_data = pd.DataFrame(response.json())
else:
    print(f"\n\nFailed to retrieve match data: {response.status_code}\n\n response: {response}\n\n")
print(response.json)

In [ ]:
con.execute("""
            CREATE TABLE match_holdout AS
SELECT *
FROM match_players
WHERE match_id IN (
    SELECT match_id
    FROM match_players
    GROUP BY match_id
    ORDER BY RANDOM()
    LIMIT 5000  -- or however many you want to hold out
);""")